In [1]:
import pandas as pd
import os
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc
from datetime import datetime
from scipy import sparse
import xlrd
from stats_can import StatsCan

In [2]:
# Option d'affichage
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', None)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', None)

In [3]:
# Options d'affchage pour les graphiques
rc('font', **{'family': 'serif', 'serif': ['Palatino']})
rc('text', usetex=True)
palette = ['#002855', '#26d07c', '#ff585d', '#f3d03e', '#0072ce', '#eb6fbd', '#00aec7', '#888b8d']

# Value added

### 1947 à 1996

In [4]:
# 1947 à 1996
value_added_1 = os.path.join(
    Path(os.getcwd()).parent,
    "Data",
    "value_added_1947.xlsx"
)

df_v1 = pd.read_excel(value_added_1, header = 5)

# Changer le nom de colonne
df_v1.columns.values[1] = "industry"
df_v1["industry"] = df_v1["industry"].str.strip()

# Dictionnaire python pour ajotuer les NAICS
naics = {
    "Farms": "111-112",
    "Forestry, fishing, and related activities": "113-115",
    "Utilities": "221",
    "Construction": "23",
    "Wood products": "321",
    "Nonmetallic mineral products": "327",
    "Primary metals": "331",
    "Fabricated metal products": "332",
    "Machinery": "333",
    "Computer and electronic products": "334",
    "Electrical equipment, appliances, and components": "335",
    "Motor vehicles, bodies and trailers, and parts": "336",
    "Furniture and related products": "337",
    "Miscellaneous manufacturing": "339",
    "Food and beverage and tobacco products": "311-312",
    "Textile mills and textile product mills": "313-314",
    "Apparel and leather and allied products": "315-316",
    "Paper products": "322",
    "Printing and related support activities": "323",
    "Petroleum and coal products": "324",
    "Chemical products": "325",
    "Plastics and rubber products": "326",
    "Wholesale trade": "41",
    "Retail trade": "44-45",
    "Transportation and warehousing": "48-49",
    "Information": "51",
    "Finance, insurance, real estate, rental, and leasing": "52-53",
    "Professional, scientific, and technical services": "54",
    "Administrative and waste management services": "56",
    "Health care and social assistance": "62",
    "Hospitals": "622",
    "Arts, entertainment, and recreation": "71",
    "Accommodation and food services": "72",
    "Other services, except government": "81"
}

# Ajouter les NAICS
df_v1["naics"] = df_v1["industry"].map(naics)


# Enlever les industries et sous industries inutiles
# ATTENTION : différences avec l'analyse industrielle canadienne
#   1. 113 et 114 sont aggrégé et deviennent : 113-114
#   2. Idem pour 311-312
#   3. Il n'y a pas l'industrie 115 dans les donneés américaines
df_v1 = df_v1.dropna(subset=["naics"])

# Remplacer les NaN par 0
df_v1 = df_v1.replace('---', 0)

# Enlever hospital de 622 (hospital) de 62
cols = [col for col in df_v1.columns if col not in ["naics", "industry"]]

# S'assurer que ces colonnes sont numériques (convertit si besoin)
for col in cols:
    df_v1[col] = pd.to_numeric(df_v1[col], errors='coerce')

if not ((df_v1["naics"] == "62").any() and (df_v1["naics"] == "622").any()):
    raise ValueError("La ligne avec naics '62' ou '622' est manquante dans le DataFrame.")


ligne_62 = df_v1[df_v1["naics"] == "62"][cols].values[0]
ligne_622 = df_v1[df_v1["naics"] == "622"][cols].values[0]
nouvelle_ligne_valeurs = ligne_62 - ligne_622
nouvelle_ligne = pd.DataFrame([nouvelle_ligne_valeurs], columns=cols)
df_v1 = df_v1[~df_v1["naics"].isin(["62", "622"])]
nouvelle_ligne.insert(0, "naics", "62")  # Ajouter la colonne 'naics' en première position
nouvelle_ligne.insert(0, "industry", "Health care and social assistance (except hospitals)")

# Ajouter la nouvelle ligne 62 au DataFrame
top = df_v1.iloc[:29]
bottom = df_v1.iloc[29:]
df_v1 = pd.concat([top, nouvelle_ligne, bottom], ignore_index=True)

# Mettre la colonne "naics" à la deuxième colonne
cols = list(df_v1.columns)
cols.insert(1, cols.pop(cols.index("naics")))
df_v1 = df_v1[cols]

# Reset l'index
df_v1 = df_v1.reset_index(drop=True)

# Drop les colonnes inutiles
df_v1 = df_v1.drop(columns = "Line")

c:\Users\edwar\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\edwar\AppData\Local\Temp\ipykernel_15852\821021901.py:64: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_v1 = df_v1.replace('---', 0)


### 1996 à 2019

In [5]:
# 1947 à 1996
value_added_2 = os.path.join(
    Path(os.getcwd()).parent,
    "Data",
    "value_added_1997.xlsx"
)


df_v2 = pd.read_excel(value_added_2, header = 5)

# Changer le nom de colonne
df_v2.columns.values[1] = "industry"
df_v2["industry"] = df_v2["industry"].str.strip()

# Ajouter les NAICS
df_v2["naics"] = df_v2["industry"].map(naics)


# Enlever le industries et sous industries inutiles
# ATTENTION : différences avec l'analyse canadienne
#   1. 113 et 114 sont aggrégé et deviennent : 113-114
#   2. Idem pour 311-312
#   3. Il n'y a pas l'industrie 115 dans les donneés américaines
df_v2 = df_v2.dropna(subset=["naics"])

# Remplacer les NaN par 0
df_v2 = df_v2.replace('---', 0)

# Enlever hospital de 622 (hospital) de 62
cols = [col for col in df_v2.columns if col not in ["naics", "industry"]]

# S'assurer que ces colonnes sont numériques (convertit si besoin)
for col in cols:
    df_v2[col] = pd.to_numeric(df_v2[col], errors='coerce')

if not ((df_v2["naics"] == "62").any() and (df_v2["naics"] == "622").any()):
    raise ValueError("La ligne avec naics '62' ou '622' est manquante dans le DataFrame.")


ligne_62 = df_v2[df_v2["naics"] == "62"][cols].values[0]
ligne_622 = df_v2[df_v2["naics"] == "622"][cols].values[0]
nouvelle_ligne_valeurs = ligne_62 - ligne_622
nouvelle_ligne = pd.DataFrame([nouvelle_ligne_valeurs], columns=cols)
df_v2 = df_v2[~df_v2["naics"].isin(["62", "622"])]
nouvelle_ligne.insert(0, "naics", "62")  # Ajouter la colonne 'naics' en première position
nouvelle_ligne.insert(0, "industry", "Health care and social assistance (except hospitals)")

# Ajouter la nouvelle ligne 62 au DataFrame
top = df_v2.iloc[:29]
bottom = df_v2.iloc[29:]
df_v2 = pd.concat([top, nouvelle_ligne, bottom], ignore_index=True)

# Mettre la colonne "naics" à la deuxième colonne
cols = list(df_v2.columns)
cols.insert(1, cols.pop(cols.index("naics")))
df_v2 = df_v2[cols]

# Reset l'index
df_v2 = df_v2.reset_index(drop=True)

# Drop les colonnes inutiles
df_v2 = df_v2.drop(columns = "Line")


### Dataframe final (1947 à 2019)

In [6]:
# Supprimer les colonnes identiques ('industry' et 'naics') dans df_v2
df_v2 = df_v2.drop(columns=["naics", "industry"])

# Fusionner horizontalement les deux DataFrames
df_value_added = pd.concat([df_v1, df_v2], axis=1)

# Transformer en format longue
id_vars = ["naics", "industry"]
value_vars = [col for col in df_value_added.columns if col not in id_vars]
df_value_added = pd.melt(
    df_value_added,
    id_vars=id_vars,
    value_vars=value_vars,
    var_name="year",
    value_name="va"
)

# Real value added

### 1947 à 1996

In [7]:
# 1947 à 1996
real_added_1 = os.path.join(
    Path(os.getcwd()).parent,
    "Data",
    "real_1947.xlsx"
)

df_r1 = pd.read_excel(real_added_1, header = 5)

# Changer le nom de colonne
df_r1.columns.values[1] = "industry"
df_r1["industry"] = df_r1["industry"].str.strip()

# Ajouter les NAICS
df_r1["naics"] = df_r1["industry"].map(naics)


# Enlever le industries et sous industries inutiles
# ATTENTION : différences avec l'analyse canadienne
#   1. 113 et 114 sont aggrégé et deviennent : 113-114
#   2. Idem pour 311-312
#   3. Il n'y a pas l'industrie 115 dans les donneés américaines
df_r1 = df_r1.dropna(subset=["naics"])

# Remplacer les NaN par 0
df_r1 = df_r1.replace('---', 0)

# Enlever hospital de 622 (hospital) de 62
cols = [col for col in df_r1.columns if col not in ["naics", "industry"]]

# S'assurer que ces colonnes sont numériques (convertit si besoin)
for col in cols:
    df_r1[col] = pd.to_numeric(df_r1[col], errors='coerce')

if not ((df_r1["naics"] == "62").any() and (df_r1["naics"] == "622").any()):
    raise ValueError("La ligne avec naics '62' ou '622' est manquante dans le DataFrame.")


ligne_62 = df_r1[df_r1["naics"] == "62"][cols].values[0]
ligne_622 = df_r1[df_r1["naics"] == "622"][cols].values[0]
nouvelle_ligne_valeurs = ligne_62 - ligne_622
nouvelle_ligne = pd.DataFrame([nouvelle_ligne_valeurs], columns=cols)
df_r1 = df_r1[~df_r1["naics"].isin(["62", "622"])]
nouvelle_ligne.insert(0, "naics", "62")  # Ajouter la colonne 'naics' en première position
nouvelle_ligne.insert(0, "industry", "Health care and social assistance (except hospitals)")

# Ajouter la nouvelle ligne 62 au DataFrame
top = df_r1.iloc[:29]
bottom = df_r1.iloc[29:]
df_r1 = pd.concat([top, nouvelle_ligne, bottom], ignore_index=True)

# Mettre la colonne "naics" à la deuxième colonne
cols = list(df_r1.columns)
cols.insert(1, cols.pop(cols.index("naics")))
df_r1 = df_r1[cols]

# Reset l'index
df_r1 = df_r1.reset_index(drop=True)

# Drop les colonnes inutiles
df_r1 = df_r1.drop(columns = "Line")

c:\Users\edwar\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\edwar\AppData\Local\Temp\ipykernel_15852\3497964394.py:26: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_r1 = df_r1.replace('---', 0)


### 1997 à 2019

In [8]:
# 1947 à 1996
real_added_2 = os.path.join(
    Path(os.getcwd()).parent,
    "Data",
    "real_1997.xlsx"
)


df_r2 = pd.read_excel(real_added_2, header = 5)

# Changer le nom de colonne
df_r2.columns.values[1] = "industry"
df_r2["industry"] = df_r2["industry"].str.strip()

# Ajouter les NAICS
df_r2["naics"] = df_r2["industry"].map(naics)


# Enlever le industries et sous industries inutiles
# ATTENTION : différences avec l'analyse canadienne
#   1. 113 et 114 sont aggrégé et deviennent : 113-114
#   2. Idem pour 311-312
#   3. Il n'y a pas l'industrie 115 dans les donneés américaines
df_r2 = df_r2.dropna(subset=["naics"])

# Remplacer les NaN par 0
df_r2 = df_r2.replace('---', 0)

# Enlever hospital de 622 (hospital) de 62
cols = [col for col in df_r2.columns if col not in ["naics", "industry"]]

# S'assurer que ces colonnes sont numériques (convertit si besoin)
for col in cols:
    df_r2[col] = pd.to_numeric(df_r2[col], errors='coerce')

if not ((df_r2["naics"] == "62").any() and (df_r2["naics"] == "622").any()):
    raise ValueError("La ligne avec naics '62' ou '622' est manquante dans le DataFrame.")


ligne_62 = df_r2[df_r2["naics"] == "62"][cols].values[0]
ligne_622 = df_r2[df_r2["naics"] == "622"][cols].values[0]
nouvelle_ligne_valeurs = ligne_62 - ligne_622
nouvelle_ligne = pd.DataFrame([nouvelle_ligne_valeurs], columns=cols)
df_r2 = df_r2[~df_r2["naics"].isin(["62", "622"])]
nouvelle_ligne.insert(0, "naics", "62")  # Ajouter la colonne 'naics' en première position
nouvelle_ligne.insert(0, "industry", "Health care and social assistance (except hospitals)")

# Ajouter la nouvelle ligne 62 au DataFrame
top = df_r2.iloc[:29]
bottom = df_r2.iloc[29:]
df_r2 = pd.concat([top, nouvelle_ligne, bottom], ignore_index=True)

# Mettre la colonne "naics" à la deuxième colonne
cols = list(df_r2.columns)
cols.insert(1, cols.pop(cols.index("naics")))
df_r2 = df_r2[cols]

# Reset l'index
df_r2 = df_r2.reset_index(drop=True)

# Drop les colonnes inutiles
df_r2 = df_r2.drop(columns = "Line")

c:\Users\edwar\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


### Dataframe final (1947 à 2019)

In [9]:
# Supprimer les colonnes identiques ('industry' et 'naics') dans df_r2
df_r2 = df_r2.drop(columns=["industry", "naics"])

# Fusionner horizontalement les deux DataFrames
df_real = pd.concat([df_r1, df_r2], axis=1)

# Transformer en format longue
value_vars = [col for col in df_real.columns if col not in id_vars]
df_real = pd.melt(
    df_real,
    id_vars=id_vars,
    value_vars=value_vars,
    var_name="year",
    value_name="real_va"
)

# Dataframe value added et real valued added

In [10]:
df = pd.merge(
    df_value_added,
    df_real,
    on=["naics", "industry", "year"],
    how="inner"  # ou "outer" si tu veux garder les années même absentes
)

# Productivité

In [11]:
fichier_tfp = os.path.join(
    Path(os.getcwd()).parent,
    "Data",
    "tfp.xlsx"
)

df_tfp = pd.read_excel(fichier_tfp, sheet_name = "MachineReadable")

In [12]:
# Harmoniser avec les dataframes de value added
dic_tfp = {
    "22": "221",
    "3361-3363": "336",
    "42": "41",
    "44,45": "44-45",
    "622-623": "622"
}

df_tfp["NAICS"] = df_tfp["NAICS"].replace(dic_tfp)
df_tfp = df_tfp.rename(columns={
    "NAICS": "naics"
    })

# Les industries à garder
liste_valeurs = [
    "111-112",
    "113-115",
    "221",
    "23",
    "321",
    "327",
    "331",
    "332",
    "333",
    "334",
    "335",
    "336",
    "337",
    "339",
    "311-312",
    "313-314",
    "315-316",
    "322",
    "323",
    "324",
    "325",
    "326",
    "41",
    "44-45",
    "48-49",
    "51",
    "52-53",
    "54",
    "56",
    "62",
    "71",
    "72",
    "81"
]

# Garder ces industries seulement
df_tfp = df_tfp[df_tfp["naics"].isin(liste_valeurs)]

# Garder les variables utiles 
variables_a_garder = [
    "Total factor productivity",
    "Labor input",
    "Capital input",
    "Labor costs",
    "Hours worked",
    "Capital costs"
]

# Enlever les éléments inutiles du dataframe
df_tfp = df_tfp[df_tfp["Measure"].isin(variables_a_garder)]
df_tfp = df_tfp.drop(columns = ["Basis", "Units"])

# Enlever les NaN de la colonne "Value"
df_tfp = df_tfp[df_tfp["Value"] != "N.A."]

# Créer des colones pour nos variables (mesures) pertinentes
df_tfp = df_tfp.pivot_table(
    index=["naics", "Industry", "Year"],  # Identifiants uniques
    columns="Measure",                    # Valeurs qui deviendront des colonnes
    values="Value",                       # Valeurs à placer dans les nouvelles colonnes
    aggfunc="first"                       # Si doublons, prendre la première valeur
).reset_index()

# Renommer les colonnes
df_tfp = df_tfp.rename(columns={
    "Year": "year",
    "Industry": "industry",
    "Total factor productivity": "tfp",
    "Labor input": "labor",
    "Capital input": "capital",
    "Labor costs": "labor_cost",
    "Hours worked": "hours",
    "Capital costs": "capital_cost",
    })

# Harmoniser la colonnes "industry"
industry = {
    "Crop & animal production (Farms)": "Farms",
    "Primary metal products": "Primary metals",
    "Apparel and leather and applied products": "Apparel and leather and allied products",
    "Finance, insurance, real estate, and leasing": "Finance, insurance, real estate, rental, and leasing",
    "Health care and social assistance": "Health care and social assistance (except hospitals)"
}

df_tfp["industry"] = df_tfp["industry"].replace(industry)

# Fusionner les deux datagframe
df_tfp["year"] = df_tfp["year"].astype(int)
df["year"] = df["year"].astype(int)

df = df_tfp.merge(
    df,
    on=["naics", "year", "industry"],
    how="left"
)

# Garder les données avant 2020
df = df[df['year'] < 2020]

# Liste des colonnes à convertir en numérique
colonnes_numeriques = ['tfp', 'capital', 'labor', 'va', 'real_va', 'capital_cost', 'labor_cost']

# Appliquer la conversion avec gestion des erreurs
for col in colonnes_numeriques:
    df[col] = pd.to_numeric(df[col], errors='coerce')

In [13]:
for i in df["industry"].unique():
    print(i)

Farms
Forestry, fishing, and related activities
Utilities
Construction
Food and beverage and tobacco products
Textile mills and textile product mills
Apparel and leather and allied products
Wood products
Paper products
Printing and related support activities
Petroleum and coal products
Chemical products
Plastics and rubber products
Nonmetallic mineral products
Primary metals
Fabricated metal products
Machinery
Computer and electronic products
Electrical equipment, appliances, and components
Motor vehicles, bodies and trailers, and parts
Furniture and related products
Miscellaneous manufacturing
Wholesale trade
Retail trade
Transportation and warehousing
Information
Finance, insurance, real estate, rental, and leasing
Professional, scientific, and technical services
Administrative and waste management services
Health care and social assistance (except hospitals)
Arts, entertainment, and recreation
Accommodation and food services
Other services, except government


In [14]:
for i in df["naics"].unique():
    print(i)

111-112
113-115
221
23
311-312
313-314
315-316
321
322
323
324
325
326
327
331
332
333
334
335
336
337
339
41
44-45
48-49
51
52-53
54
56
62
71
72
81


In [15]:
df["naics"].nunique()

33

# Calcul des variables de la productivité

In [16]:
# Rescale the variables to 1987=100
df['tfp'] = df['tfp'] / df.loc[df['year'] == 1987, 'tfp'].values[0] * 100
df['real_va'] = df['real_va'] / df.loc[df['year'] == 1987, 'real_va'].values[0] * 100
df['capital'] = df['capital'] / df.loc[df['year'] == 1987, 'capital'].values[0] * 100
df['labor'] = df['labor'] / df.loc[df['year'] == 1987, 'labor'].values[0] * 100

In [17]:
# Initialiser la colonne tfp_adj avec des NaN
df['tfp_adj'] = np.nan

# Définir la valeur de départ pour 1987
df.loc[df['year'] == 1987, 'tfp_adj'] = 100

# Boucler sur chaque industrie
for industry in df['industry'].unique():
    for year in range(1988, 2020):
        # Extraire les valeurs pour l’année courante et l’année précédente
        current = df[(df['year'] == year) & (df['industry'] == industry)]
        previous = df[(df['year'] == year - 1) & (df['industry'] == industry)]

        # Sauter si une des deux années est manquante
        if current.empty or previous.empty:
            continue

        # Calcul du poids du capital (alpha)
        cap_cost_now = current['capital_cost'].iloc[0]
        lab_cost_now = current['labor_cost'].iloc[0]
        cap_cost_prev = previous['capital_cost'].iloc[0]
        lab_cost_prev = previous['labor_cost'].iloc[0]

        alpha_now = cap_cost_now / (cap_cost_now + lab_cost_now)
        alpha_prev = cap_cost_prev / (cap_cost_prev + lab_cost_prev)
        alpha = 0.5 * (alpha_now + alpha_prev)

        # Valeurs nécessaires pour la formule
        tfp_prev = previous['tfp_adj'].iloc[0]
        real_va_now = current['real_va'].iloc[0]
        real_va_prev = previous['real_va'].iloc[0]
        capital_now = current['capital'].iloc[0]
        capital_prev = previous['capital'].iloc[0]
        labor_now = current['labor'].iloc[0]
        labor_prev = previous['labor'].iloc[0]

        # Calcul du TFP ajusté
        growth_va = np.log(real_va_now / real_va_prev)
        growth_cap = np.log((capital_now / real_va_now) / (capital_prev / real_va_prev))
        growth_lab = np.log(labor_now / labor_prev)

        tfp_adj = tfp_prev * np.exp(growth_va - (alpha / (1 - alpha)) * growth_cap - growth_lab)

        # Assigner le résultat
        df.loc[(df['year'] == year) & (df['industry'] == industry), 'tfp_adj'] = tfp_adj


C:\Users\edwar\AppData\Local\Temp\ipykernel_15852\2121794548.py:38: RuntimeWarning: invalid value encountered in log
  growth_va = np.log(real_va_now / real_va_prev)
C:\Users\edwar\AppData\Local\Temp\ipykernel_15852\2121794548.py:39: RuntimeWarning: invalid value encountered in log
  growth_cap = np.log((capital_now / real_va_now) / (capital_prev / real_va_prev))
C:\Users\edwar\AppData\Local\Temp\ipykernel_15852\2121794548.py:38: RuntimeWarning: divide by zero encountered in log
  growth_va = np.log(real_va_now / real_va_prev)
C:\Users\edwar\AppData\Local\Temp\ipykernel_15852\2121794548.py:39: RuntimeWarning: divide by zero encountered in scalar divide
  growth_cap = np.log((capital_now / real_va_now) / (capital_prev / real_va_prev))
C:\Users\edwar\AppData\Local\Temp\ipykernel_15852\2121794548.py:39: RuntimeWarning: invalid value encountered in log
  growth_cap = np.log((capital_now / real_va_now) / (capital_prev / real_va_prev))
C:\Users\edwar\AppData\Local\Temp\ipykernel_15852\212179

In [18]:
# Calculate the share of value-added of each industry within year
df['va_agg'] = df.groupby('year')['va'].transform('sum')
df['b'] = df['va'] / df['va_agg']
df['b'] = df.groupby('industry')['b'].transform(lambda x: x.rolling(2).mean())
df = df.drop(columns=['va_agg'])

# Calculate the share of value-added of each industry for years 1987 and 2000
df = pd.merge(df, df.loc[df['year'] == 1988, ['industry', 'b']].rename(columns={'b': 'b_1987'}), on='industry', how='left')
df = pd.merge(df, df.loc[df['year'] == 2000, ['industry', 'b']].rename(columns={'b': 'b_2000'}), on='industry', how='left')

# Calculate the log difference of TFP, capital, and labor within each industry
df['tfp_growth'] = df.groupby('industry')['tfp'].transform(lambda x: np.log(x).diff())
df['tfp_adj_growth'] = df.groupby('industry')['tfp'].transform(lambda x: np.log(x).diff())
df['capital_growth'] = df.groupby('industry')['capital'].transform(lambda x: np.log(x).diff())
df['labor_growth'] = df.groupby('industry')['labor'].transform(lambda x: np.log(x).diff())

# Calculate the industry-level output elasticities of capital and labor
df['alpha_k'] = df['capital_cost'] / (df['capital_cost'] + df['labor_cost'])
df['alpha_k'] = df.groupby('industry')['alpha_k'].transform(lambda x: x.rolling(2).mean())
df['alpha_l'] = df['labor_cost'] / (df['capital_cost'] + df['labor_cost'])
df['alpha_l'] = df.groupby('industry')['alpha_l'].transform(lambda x: x.rolling(2).mean())

# Calculate the share of total labor and capital costs of each industry within year
df['capital_cost_agg'] = df.groupby('year')['capital_cost'].transform('sum')
df['omega_k'] = df['capital_cost'] / df['capital_cost_agg']
df['omega_k'] = df.groupby('industry')['omega_k'].transform(lambda x: x.rolling(2).mean())
df['labor_cost_agg'] = df.groupby('year')['labor_cost'].transform('sum')
df['omega_l'] = df['labor_cost'] / df['labor_cost_agg']
df['omega_l'] = df.groupby('industry')['omega_l'].transform(lambda x: x.rolling(2).mean())
df = df.drop(columns=['capital_cost_agg', 'labor_cost_agg'])

In [19]:
df

,naics,industry,year,capital_cost,capital,hours,labor_cost,labor,tfp,va,real_va,tfp_adj,b,b_1987,b_2000,tfp_growth,tfp_adj_growth,capital_growth,labor_growth,alpha_k,alpha_l,omega_k,omega_l
0,111-112,Farms,1987,57.960,100.000000,116.821,20.262,100.000000,100.000000,62.0,100.000000,100.000000,NaN,0.015176,0.009221,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,111-112,Farms,1988,53.826,105.000000,115.818,20.873,99.194156,92.798010,61.4,88.915581,57.080612,0.015176,0.015176,0.009221,-0.074745,-0.074745,0.048790,-0.008091,0.730770,0.269230,0.050812,0.008987
2,111-112,Farms,1989,60.486,106.419349,115.246,22.637,98.807499,98.407600,73.9,101.024984,87.877097,0.015328,0.015176,0.009221,0.058693,0.058693,0.013427,-0.003906,0.724120,0.275880,0.047252,0.008876
3,111-112,Farms,1990,59.841,104.443301,107.761,24.064,92.699068,104.178542,77.8,109.038010,129.163668,0.016161,0.015176,0.009221,0.056988,0.056988,-0.018743,-0.063815,0.720434,0.279566,0.046008,0.009053
4,111-112,Farms,1991,52.002,102.480352,107.464,23.207,93.057840,105.291412,70.4,109.655491,137.132888,0.015189,0.015176,0.009221,0.010626,0.010626,-0.018973,0.003863,0.702316,0.297684,0.042087,0.008801
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1084,81,"Other services, except government",2015,28.005,90.938436,92.564,270.380,85.558933,152.050064,403.5,174.156524,76.869150,0.027840,0.031246,0.033590,0.001161,0.001161,0.011091,0.040448,0.093201,0.906799,0.005813,0.036388
1085,81,"Other services, except government",2016,28.785,92.616954,95.51,285.821,88.510907,151.172435,415.9,174.293544,74.231079,0.027770,0.031246,0.033590,-0.005789,-0.005789,0.018289,0.033920,0.092675,0.907325,0.005892,0.037011
1086,81,"Other services, except government",2017,33.855,93.562874,100,300.171,92.946305,150.795446,433.2,177.948608,72.252794,0.027722,0.031246,0.033590,-0.002497,-0.002497,0.010161,0.048896,0.096425,0.903575,0.006258,0.037509
1087,81,"Other services, except government",2018,38.076,94.251497,102.807,315.074,96.093467,150.973385,457.7,183.658979,72.333711,0.027744,0.031246,0.033590,0.001179,0.001179,0.007333,0.033299,0.104586,0.895414,0.006847,0.037625


# Préparer les tableaux de IO

### 1947 à 1962

### 1963 à 1996

In [20]:
# Dictionnaire pour mapper les industries des donneés de IO tables
naics_mapping = {
    "111CA": "111-112",
    "113FF": "113-115",
    "22": "221",
    "23": "23",
    "321": "321",
    "327": "327",
    "331": "331",
    "332": "332",
    "333": "333",
    "334": "334",
    "335": "335",
    "3361MV": "336",
    "3364OT": "336",
    "337": "337",
    "339": "339",
    "311FT": "311-312",
    "313TT": "313-314",
    "315AL": "315-316",
    "322": "322",
    "323": "323",
    "324": "324",
    "325": "325",
    "326": "326",
    "42": "41",
    "44RT": "44-45",
    "441": "44-45", # Pour données 1997-2019
    "445": "44-45", # Pour données 1997-2019
    "452": "44-45", # Pour données 1997-2019
    "4A0": "44-45", # Pour données 1997-2019
    "481": "48-49",
    "482": "48-49",
    "483": "48-49",
    "484": "48-49",
    "485": "48-49",
    "486": "48-49",
    "487OS": "48-49",
    "493": "48-49",
    "511": "51",
    "512": "51",
    "513": "51",
    "514": "51",
    "521CI": "52-53",
    "523": "52-53",
    "524": "52-53",
    "525": "52-53",
    "531": "52-53",
    "532RL": "52-53",
    "5411": "54",
    "5415": "54",
    "5412OP": "54",
    "561": "56",
    "562": "56",
    "621": "62",
    #"622HO": "622", # À enlever de 62 (hospital)
    "624": "62",
    "711AS": "71",
    "713": "71",
    "721": "72",
    "722": "72",
    "81": "81"
}

# Le chemin pour fichier excel
io_1963 = os.path.join(
    Path(os.getcwd()).parent,
    "Data",
    "IO 1963-1996.xlsx"
)

# Créer le Dataframe vide
df_io_63_96 = pd.DataFrame()

# Itération pour chaque année
for year in range(1963, 1996 + 1):
    df_io = pd.read_excel(io_1963, sheet_name = str(year), header = 6)

    # Enlever les colones inutles et renommer les colonnes
    df_io = df_io.drop(columns = ["Industry Description"])
    df_io = df_io.rename(columns={ "Code": "supply_code_agg"})

    # Ajouter colonne année
    df_io["year"] = (year)

    df_io = df_io.melt(
        id_vars=["supply_code_agg", "year"],
        var_name="use_code_agg",
        value_name="value"
        )

    # Mapper les codes selon le dictionnaire
    df_io["supply_code_agg"] = df_io["supply_code_agg"].map(naics_mapping)
    df_io["use_code_agg"] = df_io["use_code_agg"].map(naics_mapping)

    # Supprimer lignes avec NaN (non mappées)
    df_io = df_io.dropna(subset=["supply_code_agg", "use_code_agg"])

    # Enlever les lignes où "value" est "..."
    df_io["value"] = df_io["value"].replace ("...", 0)

    # Additioner les industries aggrégées
    df_io = df_io.groupby(['supply_code_agg', 'use_code_agg', 'year'])['value'].sum().reset_index()


    # Mettre les colonnes dans la bonne ordre
    df_io= df_io[["supply_code_agg", "use_code_agg", "year", "value"]]

    # Ajouter capital costs
    df_capital = df.loc[df['year'] == (year), ['capital_cost', 'naics']].rename(columns={'naics': 'use_code_agg'})
    df_capital['supply_code_agg'] = 'capital'
    df_capital['capital_cost'] = df_capital['capital_cost'] * 1000
    df_io = pd.merge(df_io, df_capital, on=['use_code_agg', 'supply_code_agg'], how='left')
    df_io.loc[
        (df_io['supply_code_agg'] == 'capital') & ~df_io['use_code_agg'].isin(['capital', 'labor']),
        'value'
    ] = df_io.loc[
        (df_io['supply_code_agg'] == 'capital') & ~df_io['use_code_agg'].isin(['capital', 'labor']),
        'capital_cost'
    ]
    df_io = df_io.drop(columns=['capital_cost'])


    # Ajouter labor costs
    df_labor = df.loc[df['year'] == (year), ['labor_cost', 'naics']].rename(columns={'naics': 'use_code_agg'})
    df_labor['supply_code_agg'] = 'labor'
    df_labor['labor_cost'] = df_labor['labor_cost'] * 1000
    df_io = pd.merge(df_io, df_labor, on=['supply_code_agg', 'use_code_agg'], how='left')
    df_io.loc[
        (df_io['supply_code_agg'] == 'labor') & ~df_io['use_code_agg'].isin(['capital', 'labor']),
        'value'
    ] = df_io.loc[
        (df_io['supply_code_agg'] == 'labor') & ~df_io['use_code_agg'].isin(['capital', 'labor']),
        'labor_cost'
    ]
    df_io = df_io.drop(columns=['labor_cost'])

    # Fill in the missing values with 0
    df_io.loc[df_io['value'].isna(), 'value'] = 0

    # Calculate the cost share of each industry
    df_io['cost_share'] = df_io.groupby('use_code_agg')['value'].transform(lambda x: x / x.sum())
    df_io.loc[df_io['cost_share'].isna(), 'cost_share'] = 0

    # Sort the data frame by year, use_code_agg, and supply_code_agg
    df_io = df_io.sort_values(by=['use_code_agg', 'supply_code_agg'])

    # Append the data to the Dataframe
    df_io_63_96 = pd.concat([df_io_63_96, df_io], ignore_index=True)

# Sort the data frame by year, use_code_agg, and supply_code_agg
df_io_63_96 = df_io_63_96.sort_values(by=['year', 'use_code_agg', 'supply_code_agg'])

C:\Users\edwar\AppData\Local\Temp\ipykernel_15852\3993801291.py:100: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_io["value"] = df_io["value"].replace ("...", 0)
C:\Users\edwar\AppData\Local\Temp\ipykernel_15852\3993801291.py:100: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_io["value"] = df_io["value"].replace ("...", 0)
C:\Users\edwar\AppData\Local\Temp\ipykernel_15852\3993801291.py:100: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, exp

### 1997 à 2019


In [21]:
# Créer le Dataframe vide
df_io_97_19 = pd.DataFrame()

# Itération pour les années 1997 à 2019
for year in range(1997, 2019 + 1):
    df_io = pd.read_excel(
        os.path.join(Path(os.getcwd()).parent, 
                     "Data",
                     "Table " + str(year) + ".xlsx"), 
                     header= 5
    )
    
    # Enlever les colonnes inutiles et renommer
    df_io = df_io.drop(columns = ["Industries/Industries"])
    df_io = df_io.rename(columns={ "Unnamed: 0": "supply_code_agg"})
    df_io = df_io.drop(index = 0)
    
    # Ajouter colonne année
    df_io["year"] = (year)

    df_io = df_io.melt(
        id_vars=["supply_code_agg", "year"],
        var_name="use_code_agg",
        value_name="value"
        )

    # Mapper les codes selon le dictionnaire
    df_io["supply_code_agg"] = df_io["supply_code_agg"].map(naics_mapping)
    df_io["use_code_agg"] = df_io["use_code_agg"].map(naics_mapping)

    # Supprimer lignes avec NaN (non mappées)
    df_io = df_io.dropna(subset=["supply_code_agg", "use_code_agg"])

    # Enlever les lignes où "value" est "..."
    df_io["value"] = df_io["value"].replace ("...", 0)

    # Additioner les industries aggrégées
    df_io = df_io.groupby(['supply_code_agg', 'use_code_agg', 'year'])['value'].sum().reset_index()


    # Mettre les colonnes dans la bonne ordre
    df_io= df_io[["supply_code_agg", "use_code_agg", "year", "value"]]

    # Ajouter capital costs
    df_capital = df.loc[df['year'] == (year), ['capital_cost', 'naics']].rename(columns={'naics': 'use_code_agg'})
    df_capital['supply_code_agg'] = 'capital'
    df_capital['capital_cost'] = df_capital['capital_cost'] * 1000
    df_io = pd.merge(df_io, df_capital, on=['use_code_agg', 'supply_code_agg'], how='left')
    df_io.loc[
        (df_io['supply_code_agg'] == 'capital') & ~df_io['use_code_agg'].isin(['capital', 'labor']),
        'value'
    ] = df_io.loc[
        (df_io['supply_code_agg'] == 'capital') & ~df_io['use_code_agg'].isin(['capital', 'labor']),
        'capital_cost'
    ]
    df_io = df_io.drop(columns=['capital_cost'])


    # Ajouter labor costs
    df_labor = df.loc[df['year'] == (year), ['labor_cost', 'naics']].rename(columns={'naics': 'use_code_agg'})
    df_labor['supply_code_agg'] = 'labor'
    df_labor['labor_cost'] = df_labor['labor_cost'] * 1000
    df_io = pd.merge(df_io, df_labor, on=['supply_code_agg', 'use_code_agg'], how='left')
    df_io.loc[
        (df_io['supply_code_agg'] == 'labor') & ~df_io['use_code_agg'].isin(['capital', 'labor']),
        'value'
    ] = df_io.loc[
        (df_io['supply_code_agg'] == 'labor') & ~df_io['use_code_agg'].isin(['capital', 'labor']),
        'labor_cost'
    ]
    df_io = df_io.drop(columns=['labor_cost'])

    # Fill in the missing values with 0
    df_io.loc[df_io['value'].isna(), 'value'] = 0

    # Calculate the cost share of each industry
    df_io['cost_share'] = df_io.groupby('use_code_agg')['value'].transform(lambda x: x / x.sum())
    df_io.loc[df_io['cost_share'].isna(), 'cost_share'] = 0

    # Sort the data frame by year, use_code_agg, and supply_code_agg
    df_io = df_io.sort_values(by=['use_code_agg', 'supply_code_agg'])

    # Append the data to the Dataframe
    df_io_97_19 = pd.concat([df_io_97_19, df_io], ignore_index=True)

# Sort the data frame by year, use_code_agg, and supply_code_agg
df_io_97_19 = df_io_97_19.sort_values(by=['year', 'use_code_agg', 'supply_code_agg'])

c:\Users\edwar\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\edwar\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\edwar\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\edwar\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl'

# Append the I-O tables across all years and calculate the lambda's

In [22]:
# Concatenate the data frames
df_io = pd.concat([df_io_63_96, df_io_97_19], ignore_index=True)
df_io = df_io.sort_values(by=['year', 'use_code_agg', 'supply_code_agg'])

In [ ]:
### À changer si je trouve les données avant 1987
df_io = df_io[df_io["year"] > 1986].reset_index(drop = True)

In [ ]:
# Create the cost-based IO matrices for each year


### MATRICE NON INVERSIBLE
df_lambda = pd.DataFrame({'year': df['year'].unique()})
df_lambda['lambda_k'] = np.nan
df_lambda['lambda_l'] = np.nan
for year in df_io['year'].unique():
    df_io_year = df_io[df_io['year'] == year]
    io_matrix = df_io_year.pivot(
    index='use_code_agg',
    columns='supply_code_agg',
    values='cost_share'
    ).fillna(0).astype(float).values

    io_matrix = sparse.csr_matrix(io_matrix)
    b = df.loc[df['year'] == year, ['naics', 'va']].sort_values(by=['naics'])['va'].values
    b = b / b.sum()
    b = np.append(b, [0, 0])
    lambda_tilde = np.matmul(b.transpose(), np.linalg.inv(np.eye(io_matrix.shape[0]) - io_matrix))
    df_lambda.loc[df_lambda['year'] == year, 'lambda_k'] = lambda_tilde[0, -2]
    df_lambda.loc[df_lambda['year'] == year, 'lambda_l'] = lambda_tilde[0, -1]

# Take the average of successive years
df_lambda['lambda_k'] = df_lambda['lambda_k'].rolling(2).mean()
df_lambda['lambda_l'] = df_lambda['lambda_l'].rolling(2).mean()

# Merge the lambda's with the original data
df = pd.merge(df, df_lambda[['year', 'lambda_k', 'lambda_l']], on='year', how='left')

C:\Users\edwar\AppData\Local\Temp\ipykernel_15852\3944819721.py:11: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  ).fillna(0).astype(float).values


LinAlgError: Singular matrix

In [ ]:
df.columns

Index(['naics', 'industry', 'year', 'capital_cost', 'capital', 'hours',
       'labor_cost', 'labor', 'tfp', 'va', 'real_va', 'tfp_adj', 'b', 'b_1987',
       'b_2000', 'tfp_growth', 'tfp_adj_growth', 'capital_growth',
       'labor_growth', 'alpha_k', 'alpha_l', 'omega_k', 'omega_l'],
      dtype='object')